In [ ]:
#!unzip "sfc_data.zip"

In [ ]:
#loading important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline

In [ ]:
#loading the dataset to model m1
train=pd.read_csv('sfc_data/train_new.csv')
test=pd.read_csv('sfc_data/test_new.csv')
ss=pd.read_csv('sfc_data/SampleSubmission.csv')

In [ ]:
train.head()

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,ID.1,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,lastransdate,firstransdate,delay_term_payment,m0,delay_first_payment,total_payment
0,ID_MR53LEX,2018-04-23 12:28:05,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,NaN,Coast Region,Kwale,Teacher,d_light,556,33080.0,2019-10-31 12:28:05,2018-04-23 12:28:05,2021-05-26 17:14:45,ID_MR53LEX,"['04-2018', '05-2018', '06-2018', '07-2018', '...","[3600.0, 750.0, 350.0, 65.0, 95.0, 135.0, 85.0...",880.0,930.0,495.0,715.0,220.0,385.0,2020-11-01 00:00:00,2018-04-01 00:00:00,12.0,770,0.0,16035
1,ID_3D7NQUH,2018-04-17 10:27:35,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,26.0,South Rift,Narok,Farmer,d_light,556,33080.0,2019-10-25 10:27:35,2018-04-17 10:27:35,2021-05-26 16:54:53,ID_3D7NQUH,"['04-2018', '05-2018', '06-2018', '07-2018', '...","[2940.0, 970.0, 380.0, 880.0, 385.0, 440.0, 11...",660.0,935.0,935.0,825.0,770.0,935.0,2020-11-01 00:00:00,2018-04-01 00:00:00,12.0,770,0.0,22136
2,ID_0IWQNPI,2020-02-18 14:23:44,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,21.0,Mount Kenya Region,Embu,Business,d_light,240,14400.0,2020-10-15 14:23:44,2020-02-18 14:23:48,2021-01-25 18:46:20,ID_0IWQNPI,"['02-2020', '03-2020', '04-2020', '05-2020', '...","[2850.0, 1500.0, 1350.0, 610.0, 200.0, 250.0]",700.0,1350.0,1550.0,1400.0,1450.0,1200.0,2020-07-01 00:00:00,2020-02-01 00:00:00,3.0,250,0.0,6760
3,ID_IY8SYB9,2017-09-14 11:07:40,2000,NaN,0.0,FINANCED,DAILY,40,7,Female,26.0,Mount Kenya Region,Meru,Farmer,d_light,364,16560.0,2018-09-13 11:07:40,2017-09-14 11:07:32,2018-12-20 16:50:39,ID_IY8SYB9,"['09-2017', '10-2017', '11-2017', '12-2017', '...","[2200.0, 1420.0, 1180.0, 900.0, 1400.0, 780.0,...",580.0,480.0,800.0,1260.0,1650.0,530.0,2018-06-01 00:00:00,2017-09-01 00:00:00,3.0,380,0.0,11260
4,ID_9XHL7VZ,2017-09-06 06:50:20,2000,NaN,0.0,FINANCED,DAILY,40,7,Male,27.0,North Rift,West Pokot,Farmer,d_light,364,16560.0,2018-09-05 06:50:20,2017-09-06 06:50:16,2021-05-12 14:50:01,ID_9XHL7VZ,"['09-2017', '10-2017', '11-2017', '12-2017', '...","[2640.0, 910.0, 480.0, 280.0, 200.0, 180.0, 33...",40.0,440.0,460.0,360.0,80.0,330.0,2019-11-01 00:00:00,2017-09-01 00:00:00,13.0,520,0.0,7779


In [ ]:
train.head()

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,ID.1,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,lastransdate,firstransdate,delay_term_payment,m0,delay_first_payment,total_payment
0,ID_MR53LEX,2018-04-23 12:28:05,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,NaN,Coast Region,Kwale,Teacher,d_light,556,33080.0,2019-10-31 12:28:05,2018-04-23 12:28:05,2021-05-26 17:14:45,ID_MR53LEX,"['04-2018', '05-2018', '06-2018', '07-2018', '...","[3600.0, 750.0, 350.0, 65.0, 95.0, 135.0, 85.0...",880.0,930.0,495.0,715.0,220.0,385.0,2020-11-01 00:00:00,2018-04-01 00:00:00,12.0,770,0.0,16035
1,ID_3D7NQUH,2018-04-17 10:27:35,2500,NaN,0.0,FINANCED,DAILY,55,3,Male,26.0,South Rift,Narok,Farmer,d_light,556,33080.0,2019-10-25 10:27:35,2018-04-17 10:27:35,2021-05-26 16:54:53,ID_3D7NQUH,"['04-2018', '05-2018', '06-2018', '07-2018', '...","[2940.0, 970.0, 380.0, 880.0, 385.0, 440.0, 11...",660.0,935.0,935.0,825.0,770.0,935.0,2020-11-01 00:00:00,2018-04-01 00:00:00,12.0,770,0.0,22136
2,ID_0IWQNPI,2020-02-18 14:23:44,2400,NaN,0.0,FINANCED,DAILY,50,3,Male,21.0,Mount Kenya Region,Embu,Business,d_light,240,14400.0,2020-10-15 14:23:44,2020-02-18 14:23:48,2021-01-25 18:46:20,ID_0IWQNPI,"['02-2020', '03-2020', '04-2020', '05-2020', '...","[2850.0, 1500.0, 1350.0, 610.0, 200.0, 250.0]",700.0,1350.0,1550.0,1400.0,1450.0,1200.0,2020-07-01 00:00:00,2020-02-01 00:00:00,3.0,250,0.0,6760
3,ID_IY8SYB9,2017-09-14 11:07:40,2000,NaN,0.0,FINANCED,DAILY,40,7,Female,26.0,Mount Kenya Region,Meru,Farmer,d_light,364,16560.0,2018-09-13 11:07:40,2017-09-14 11:07:32,2018-12-20 16:50:39,ID_IY8SYB9,"['09-2017', '10-2017', '11-2017', '12-2017', '...","[2200.0, 1420.0, 1180.0, 900.0, 1400.0, 780.0,...",580.0,480.0,800.0,1260.0,1650.0,530.0,2018-06-01 00:00:00,2017-09-01 00:00:00,3.0,380,0.0,11260
4,ID_9XHL7VZ,2017-09-06 06:50:20,2000,NaN,0.0,FINANCED,DAILY,40,7,Male,27.0,North Rift,West Pokot,Farmer,d_light,364,16560.0,2018-09-05 06:50:20,2017-09-06 06:50:16,2021-05-12 14:50:01,ID_9XHL7VZ,"['09-2017', '10-2017', '11-2017', '12-2017', '...","[2640.0, 910.0, 480.0, 280.0, 200.0, 180.0, 33...",40.0,440.0,460.0,360.0,80.0,330.0,2019-11-01 00:00:00,2017-09-01 00:00:00,13.0,520,0.0,7779


In [ ]:
full = pd.concat([train,test]).reset_index()

In [ ]:
def My_model(data,n):

  ###deleting LastPaymentDate ###################
  # change to date time values

  data['FirstPaymentDate']=pd.to_datetime(data['FirstPaymentDate'])
  data['RegistrationDate']=pd.to_datetime(data['RegistrationDate'])

  ####counting the number of transactions in the contract ##################"
  
  
  data['number_of_transaction']=data['PaymentsHistory'].apply(lambda x: len(ast.literal_eval(x))+ n-1)


  #########computing max , mean ,min ############

  data['maxPayment']=data['PaymentsHistory'].apply(lambda x: max(ast.literal_eval(x)))
  data['minPayment']=data['PaymentsHistory'].apply(lambda x: min(ast.literal_eval(x)))
  data['meanPayment']=0
  for i,j in enumerate(data['PaymentsHistory']):
    data.loc[i,'meanPayment']=(sum(ast.literal_eval(data.PaymentsHistory[i]))/data.loc[i,'number_of_transaction'])


  ###creating target ############################
  data['target']= data['total_payment']/data['TotalContractValue']



  ###churn variable  ############################
  cond=[(data['target']<0.6),(data['target']>=0.6)]
  val=[0,1]
  data['churn']=np.select(cond,val)


  #####some insights #############################

  data['RegisteredInLeapYear'] = data.RegistrationDate.dt.is_leap_year.astype('float')
  data['RegisteredAtMonthStart'] = data.RegistrationDate.dt.is_month_start.astype('float')
  data['RegisteredAtMonthEnd'] = data.RegistrationDate.dt.is_month_end.astype('float')
  data['FirstPaymentMonth'] = data.FirstPaymentDate.dt.month


  ############frequency encoding ##################
  col=['Deposit','AccessoryRate','PaymentMethod', 'rateTypeEntity', 'RatePerUnit', 'DaysOnDeposit',
       'MainApplicantGender', 'Region', 'Town', 'Occupation','Term', 'TotalContractValue']
  for i in col:
    data['count_'+i] = data[i].map(data[i].value_counts())
  
  ####drop some irrelevant variables ###############

  data=data.drop(columns=['ID.1','lastransdate', 'firstransdate','Region','RegistrationDate','LastPaymentDate','Town','PaymentMethod','AccessoryRate',
                            'TransactionDates', 'PaymentsHistory','UpsellDate','ExpectedTermDate','SupplierName','FirstPaymentDate','delay_first_payment'])
  

  #### some preprocessing #########################

  _rate=pd.get_dummies(data['rateTypeEntity'])
  data=pd.concat([data.drop('rateTypeEntity',axis=1),_rate],axis=1)
  _occupation=pd.get_dummies(data['Occupation'])
  data=pd.concat([data.drop('Occupation',axis=1),_occupation],axis=1)
  _gender=pd.get_dummies(data['MainApplicantGender'])
  data=pd.concat([data.drop('MainApplicantGender',axis=1),_gender],axis=1)
  data.Age.fillna(value=round(data.Age.mean()), inplace=True)

  return data

In [ ]:
def drop_index(data):
  return data.drop(columns=['ID','index'])

In [ ]:
#!pip install catboost
#!pip install lightgbm

In [ ]:
#now we perform the modelling
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics

In [ ]:
def model_train(X_train,Y_train,X_test,test,m):
  fold=KFold(n_splits=6, shuffle=True, random_state=1234)
  testsplit_store=[]
  test_store=[]
  i=1
  for train_index, test_index in fold.split(X_train, Y_train):
    x_train, x_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train, y_test = Y_train.iloc[train_index], Y_train.iloc[test_index]
    lgb = CatBoostRegressor(eval_metric='RMSE')
    lgb.fit(x_train,y_train,eval_set=[(x_train,y_train),(x_test, y_test)], verbose=500)
    pred = lgb.predict(X_test)
    test_store.append(pred)
    pred = np.mean(test_store, 0)
    test[m]=pred
  return test

In [ ]:
####setting up for the next month########
def next_month(data,m):
  data['total_payment']+=data[m]
  data['delay_term_payment']-=1
  return data

In [ ]:
############predicting m1 ##############################
full=My_model(full,1)
Y=full[:train.shape[0]]['m1']

In [ ]:
full.head()

,index,ID,Deposit,RatePerUnit,DaysOnDeposit,Age,Term,TotalContractValue,m1,m2,m3,m4,m5,m6,delay_term_payment,m0,total_payment,number_of_transaction,maxPayment,minPayment,meanPayment,target,churn,RegisteredInLeapYear,RegisteredAtMonthStart,RegisteredAtMonthEnd,FirstPaymentMonth,count_Deposit,count_AccessoryRate,count_PaymentMethod,count_rateTypeEntity,count_RatePerUnit,count_DaysOnDeposit,count_MainApplicantGender,count_Region,count_Town,count_Occupation,count_Term,count_TotalContractValue,group_iDeposit,group_iAccessoryRate,group_iRatePerUnit,group_iDaysOnDeposit,group_iAge,group_iTerm,group_iTotalContractValue,group_im0,group_itotal_payment,DAILY,MONTHLY,WEEKLY,Business,Driver/Motorbike Rider,Farmer,Government Employee,Labourer,Other,Teacher,Female,Male
0,0,ID_MR53LEX,2500,55,3,39.0,556,33080.0,880.0,930.0,495.0,715.0,220.0,385.0,12.0,770,16035,31,3600.0,55.0,517.258065,0.484734,0,0.0,0.0,0.0,4,6549,36435,37343,37168,14867,23684,23351,4671.0,1245,1830,6474,6474,2395.435614,2.214128,48.912436,4.453274,39.500000,430.782193,24741.057395,979.103017,16811.265636,1,0,0,0,0,0,0,0,0,1,0,1
1,1,ID_3D7NQUH,2500,55,3,26.0,556,33080.0,660.0,935.0,935.0,825.0,770.0,935.0,12.0,770,22136,30,2940.0,380.0,737.866667,0.669166,1,0.0,0.0,0.0,4,6549,36435,37343,37168,14867,23684,23351,4759.0,1758,11747,6474,6474,2346.738629,1.099457,47.489817,4.610545,41.721438,418.666440,23171.108169,779.397601,15195.867391,1,0,0,0,0,1,0,0,0,0,0,1
2,2,ID_0IWQNPI,2400,50,3,21.0,240,14400.0,700.0,1350.0,1550.0,1400.0,1450.0,1200.0,3.0,250,6760,6,2850.0,200.0,1126.666667,0.469444,0,1.0,0.0,0.0,2,2736,36435,37343,37168,2744,23684,23351,4073.0,870,12380,2736,2736,2338.568896,1.055116,54.261803,4.858043,38.425287,404.390515,22460.782418,822.931318,14727.334117,1,0,0,1,0,0,0,0,0,0,0,1
3,3,ID_IY8SYB9,2000,40,7,26.0,364,16560.0,580.0,480.0,800.0,1260.0,1650.0,530.0,3.0,380,11260,10,2200.0,380.0,1126.000000,0.679952,1,0.0,0.0,0.0,9,19281,36435,37343,37168,19386,13445,13992,4073.0,1656,11747,13399,13304,2346.738629,1.099457,47.489817,4.610545,41.721438,418.666440,23171.108169,779.397601,15195.867391,1,0,0,0,0,1,0,0,0,0,1,0
4,4,ID_9XHL7VZ,2000,40,7,27.0,364,16560.0,40.0,440.0,460.0,360.0,80.0,330.0,13.0,520,7779,22,2640.0,40.0,353.590909,0.469746,0,0.0,0.0,0.0,9,19281,36435,37343,37168,19386,13445,23351,5344.0,721,11747,13399,13304,2346.738629,1.099457,47.489817,4.610545,41.721438,418.666440,23171.108169,779.397601,15195.867391,1,0,0,0,0,1,0,0,0,0,0,1


In [ ]:
full=full.drop(columns=['m1','m2','m3','m4','m5','m6'])

In [ ]:
full=drop_index(full)

In [ ]:
train_1=full[:train.shape[0]]
test_1=full[train.shape[0]:]

In [ ]:
#training the model and performing predicition for m1
test=model_train(train_1,Y,test_1,test,'m1')

Learning rate set to 0.083571
0:	learn: 675.4945814	test: 675.4945814	test1: 579.8217004	best: 579.8217004 (0)	total: 56.6ms	remaining: 56.5s
500:	learn: 311.6328847	test: 311.6328847	test1: 356.9310627	best: 354.3086863 (107)	total: 3.94s	remaining: 3.92s
999:	learn: 276.4470842	test: 276.4470842	test1: 357.7880354	best: 354.3086863 (107)	total: 7.8s	remaining: 0us

bestTest = 354.3086863
bestIteration = 107

Shrink model to first 108 iterations.
Learning rate set to 0.083571
0:	learn: 659.2204650	test: 659.2204650	test1: 667.9973735	best: 667.9973735 (0)	total: 8.66ms	remaining: 8.65s
500:	learn: 310.9849911	test: 310.9849911	test1: 499.9223730	best: 497.1908286 (124)	total: 3.8s	remaining: 3.79s
999:	learn: 274.6244939	test: 274.6244939	test1: 501.4439762	best: 497.1908286 (124)	total: 7.71s	remaining: 0us

bestTest = 497.1908286
bestIteration = 124

Shrink model to first 125 iterations.
Learning rate set to 0.083571
0:	learn: 675.4254550	test: 675.4254550	test1: 578.1299266	best: 5

In [ ]:
#test.head()
###preparing for the next month m2  #######
train=next_month(train,'m1')
test=next_month(test,'m1')

In [ ]:
full = pd.concat([train,test]).reset_index()

In [ ]:
########predicting m2 ###################
full=My_model(full,2)

In [ ]:
train_2=full[:train.shape[0]]
test_2=full[train.shape[0]:]

In [ ]:
#dropping index
train_2=drop_index(train_2)
test_2=drop_index(test_2)

In [ ]:
#drop columns
Y=train_2['m2']
train_2=train_2.drop(columns=['m2','m3','m4','m5','m6'])
test_2=test_2.drop(columns=['m2','m3','m4','m5','m6'])

In [ ]:
#training the model and performing predicition for m2
test=model_train(train_2,Y,test_2,test,'m2')

Learning rate set to 0.083571
0:	learn: 602.6865450	test: 602.6865450	test1: 1487.9458909	best: 1487.9458909 (0)	total: 9.26ms	remaining: 9.25s
500:	learn: 313.1377856	test: 313.1377856	test1: 1406.9194551	best: 1406.8874142 (472)	total: 4.04s	remaining: 4.02s
999:	learn: 275.8714745	test: 275.8714745	test1: 1407.2248974	best: 1406.4842089 (646)	total: 8.08s	remaining: 0us

bestTest = 1406.484209
bestIteration = 646

Shrink model to first 647 iterations.
Learning rate set to 0.083571
0:	learn: 846.6538750	test: 846.6538750	test1: 604.9427633	best: 604.9427633 (0)	total: 9.51ms	remaining: 9.49s
500:	learn: 316.4816898	test: 316.4816898	test1: 411.1757711	best: 403.2232877 (37)	total: 4s	remaining: 3.99s
999:	learn: 279.2165508	test: 279.2165508	test1: 412.2885007	best: 403.2232877 (37)	total: 8.05s	remaining: 0us

bestTest = 403.2232877
bestIteration = 37

Shrink model to first 38 iterations.
Learning rate set to 0.083571
0:	learn: 847.6456534	test: 847.6456534	test1: 597.2325575	best: 

In [ ]:
#test.head()

In [ ]:
#test.head()
###preparing for the next month m3  #######
train=next_month(train,'m2')
test=next_month(test,'m2')

In [ ]:
full = pd.concat([train,test]).reset_index()

In [ ]:
########predicting m3 ###################
full=My_model(full,3)

In [ ]:
train_3=full[:train.shape[0]]
test_3=full[train.shape[0]:]

In [ ]:
#dropping index
train_3=drop_index(train_3)
test_3=drop_index(test_3)

In [ ]:
#drop columns
Y=train_3['m3']
train_3=train_3.drop(columns=['m3','m4','m5','m6'])
test_3=test_3.drop(columns=['m3','m4','m5','m6'])

In [ ]:
#training the model and performing predicition for m3
test=model_train(train_3,Y,test_3,test,'m3')

Learning rate set to 0.083571
0:	learn: 659.5481732	test: 659.5481732	test1: 728.6996573	best: 728.6996573 (0)	total: 9.42ms	remaining: 9.41s
500:	learn: 317.4209759	test: 317.4209759	test1: 563.2344167	best: 559.2420808 (133)	total: 4.13s	remaining: 4.12s
999:	learn: 277.1001406	test: 277.1001406	test1: 564.2347519	best: 559.2420808 (133)	total: 8.25s	remaining: 0us

bestTest = 559.2420808
bestIteration = 133

Shrink model to first 134 iterations.
Learning rate set to 0.083571
0:	learn: 666.9190056	test: 666.9190056	test1: 691.0037928	best: 691.0037928 (0)	total: 9.41ms	remaining: 9.4s
500:	learn: 317.4133094	test: 317.4133094	test1: 499.2550555	best: 495.9029471 (191)	total: 4.07s	remaining: 4.05s
999:	learn: 275.8497865	test: 275.8497865	test1: 501.7835682	best: 495.9029471 (191)	total: 8.19s	remaining: 0us

bestTest = 495.9029471
bestIteration = 191

Shrink model to first 192 iterations.
Learning rate set to 0.083571
0:	learn: 658.3020922	test: 658.3020922	test1: 735.2867137	best: 

In [ ]:
###preparing for the next month m4  #######
train=next_month(train,'m3')
test=next_month(test,'m3')

In [ ]:
full = pd.concat([train,test]).reset_index()

In [ ]:
########predicting m4 ###################
full=My_model(full,4)

In [ ]:
train_4=full[:train.shape[0]]
test_4=full[train.shape[0]:]

In [ ]:
#dropping index
train_4=drop_index(train_4)
test_4=drop_index(test_4)

In [ ]:
#drop columns
Y=train_4['m4']
train_4=train_4.drop(columns=['m4','m5','m6'])
test_4=test_4.drop(columns=['m4','m5','m6'])

In [ ]:
#training the model and performing predicition for m4
test=model_train(train_4,Y,test_4,test,'m4')

Learning rate set to 0.083571
0:	learn: 705.8386884	test: 705.8386884	test1: 611.1596900	best: 611.1596900 (0)	total: 9.83ms	remaining: 9.82s
500:	learn: 337.3176250	test: 337.3176250	test1: 380.2003946	best: 379.5998218 (348)	total: 4.39s	remaining: 4.38s
999:	learn: 289.2199912	test: 289.2199912	test1: 381.3870440	best: 379.4872216 (675)	total: 8.77s	remaining: 0us

bestTest = 379.4872216
bestIteration = 675

Shrink model to first 676 iterations.
Learning rate set to 0.083571
0:	learn: 651.2930846	test: 651.2930846	test1: 861.4700356	best: 861.4700356 (0)	total: 9.54ms	remaining: 9.53s
500:	learn: 316.4794468	test: 316.4794468	test1: 698.1702682	best: 697.3943951 (331)	total: 4.36s	remaining: 4.34s
999:	learn: 277.9713539	test: 277.9713539	test1: 698.9322035	best: 697.3943951 (331)	total: 8.67s	remaining: 0us

bestTest = 697.3943951
bestIteration = 331

Shrink model to first 332 iterations.
Learning rate set to 0.083571
0:	learn: 678.0485997	test: 678.0485997	test1: 751.4844454	best:

In [ ]:
#test.head()
###preparing for the next month m5  #######
train=next_month(train,'m4')
test=next_month(test,'m4')

In [ ]:
full = pd.concat([train,test]).reset_index()

In [ ]:
########predicting m5 ###################
full=My_model(full,5)

In [ ]:
train_5=full[:train.shape[0]]
test_5=full[train.shape[0]:]

In [ ]:
#dropping index
train_5=drop_index(train_5)
test_5=drop_index(test_5)

In [ ]:
#drop columns
Y=train_5['m5']
train_5=train_5.drop(columns=['m5','m6'])
test_5=test_5.drop(columns=['m5','m6'])

In [ ]:
#training the model and performing predicition for m4
test=model_train(train_5,Y,test_5,test,'m5')

Learning rate set to 0.083571
0:	learn: 785.7373931	test: 785.7373931	test1: 678.8992926	best: 678.8992926 (0)	total: 9.51ms	remaining: 9.5s
500:	learn: 320.3905015	test: 320.3905015	test1: 553.4056297	best: 537.0959401 (48)	total: 4.42s	remaining: 4.4s
999:	learn: 274.4404610	test: 274.4404610	test1: 557.1872194	best: 537.0959401 (48)	total: 8.91s	remaining: 0us

bestTest = 537.0959401
bestIteration = 48

Shrink model to first 49 iterations.
Learning rate set to 0.083571
0:	learn: 683.6958881	test: 683.6958881	test1: 1115.7789243	best: 1115.7789243 (0)	total: 10.1ms	remaining: 10.1s
500:	learn: 329.8020981	test: 329.8020981	test1: 964.5847384	best: 962.8385570 (375)	total: 4.29s	remaining: 4.28s
999:	learn: 286.1497956	test: 286.1497956	test1: 962.2342810	best: 962.2342810 (999)	total: 8.71s	remaining: 0us

bestTest = 962.234281
bestIteration = 999

Learning rate set to 0.083571
0:	learn: 780.3653557	test: 780.3653557	test1: 712.9158456	best: 712.9158456 (0)	total: 9.3ms	remaining: 9.

In [ ]:
###preparing for the next month m6  #######
train=next_month(train,'m5')
test=next_month(test,'m5')

In [ ]:
full = pd.concat([train,test]).reset_index()

In [ ]:
########predicting m6 ###################
full=My_model(full,6)

In [ ]:
train_6=full[:train.shape[0]]
test_6=full[train.shape[0]:]

In [ ]:
#dropping index
train_6=drop_index(train_6)
test_6=drop_index(test_6)

In [ ]:
#drop columns
Y=train_6['m6']
train_6=train_6.drop(columns=['m6'])
test_6=test_6.drop(columns=['m6'])

In [ ]:
#training the model and performing predicition for m6
test=model_train(train_6,Y,test_6,test,'m6')

Learning rate set to 0.083571
0:	learn: 1289.1505361	test: 1289.1505361	test1: 1788.5436356	best: 1788.5436356 (0)	total: 10.3ms	remaining: 10.3s
500:	learn: 573.1124294	test: 573.1124294	test1: 1665.6913430	best: 1658.7494411 (79)	total: 4.54s	remaining: 4.52s
999:	learn: 442.2105878	test: 442.2105878	test1: 1672.5074852	best: 1658.7494411 (79)	total: 9.03s	remaining: 0us

bestTest = 1658.749441
bestIteration = 79

Shrink model to first 80 iterations.
Learning rate set to 0.083571
0:	learn: 1378.4535229	test: 1378.4535229	test1: 1487.2607098	best: 1487.2607098 (0)	total: 8.66ms	remaining: 8.65s
500:	learn: 603.5290931	test: 603.5290931	test1: 1281.4725220	best: 1267.2855454 (370)	total: 4.47s	remaining: 4.45s
999:	learn: 453.4992011	test: 453.4992011	test1: 1283.6860160	best: 1267.2855454 (370)	total: 8.88s	remaining: 0us

bestTest = 1267.285545
bestIteration = 370

Shrink model to first 371 iterations.
Learning rate set to 0.083571
0:	learn: 1398.0727702	test: 1398.0727702	test1: 138

In [ ]:
train.describe()

,Deposit,AccessoryRate,RatePerUnit,DaysOnDeposit,Age,Term,TotalContractValue,m1,m2,m3,m4,m5,m6,delay_term_payment,m0,delay_first_payment,total_payment
count,28007.000000,28007.000000,28007.000000,28007.000000,22826.000000,28007.000000,28007.000000,28007.000000,28007.000000,28007.000000,28007.000000,28007.000000,28007.000000,28007.000000,28007.000000,28007.000000,28007.000000
mean,2348.559432,1.164138,49.784768,4.592566,38.924384,412.400864,22962.232763,815.411683,807.101617,797.641161,815.011033,827.719463,692.115150,1.106652,821.344985,0.018460,19139.387582
std,418.987356,7.985585,58.142391,3.336599,12.336405,121.495244,10166.829070,686.130805,839.772177,696.809281,715.582046,796.379837,1416.301221,5.911124,700.173992,0.134609,10155.133026
min,0.000000,0.000000,35.000000,0.000000,19.000000,10.000000,10800.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-5.000000,1.000000,0.000000,2455.000000
25%,2000.000000,0.000000,40.000000,3.000000,29.000000,364.000000,16560.000000,275.000000,250.000000,220.000000,230.000000,220.000000,130.000000,-3.000000,290.000000,0.000000,12370.000000
50%,2000.000000,0.000000,40.000000,3.000000,36.000000,364.000000,16560.000000,820.000000,800.000000,790.000000,815.000000,840.000000,480.000000,-1.000000,825.000000,0.000000,15840.000000
75%,2500.000000,0.000000,55.000000,7.000000,46.000000,547.000000,33080.000000,1200.000000,1200.000000,1200.000000,1200.000000,1205.000000,915.000000,3.000000,1200.000000,0.000000,29707.000000
max,8000.000000,240.000000,4000.000000,60.000000,120.000000,626.000000,64484.000000,33890.000000,94044.000000,32005.000000,33403.000000,64106.000000,96814.000000,36.000000,33237.000000,1.000000,117136.000000


In [ ]:
test.describe()

,Deposit,AccessoryRate,RatePerUnit,DaysOnDeposit,Age,Term,TotalContractValue,m0,delay_term_payment,delay_first_payment,total_payment,m1,m2,m3,m4,m5,m6
count,9336.000000,9336.000000,9336.000000,9336.000000,7578.000000,9336.000000,9336.000000,9336.000000,9336.000000,9336.000000,9336.000000,9336.000000,9336.000000,9336.000000,9336.000000,9336.000000,9336.000000
mean,2348.848436,1.051949,50.341153,4.625643,38.648720,412.508033,22925.323265,814.491324,6.845865,0.020351,19284.888620,810.269887,834.695735,823.310483,863.549382,841.314740,905.829281
std,414.298835,7.178852,55.530174,3.692646,12.191506,121.930348,10099.288427,599.611099,8.634252,0.141207,10073.269960,486.835652,465.810028,480.960773,482.780509,488.182135,724.488346
min,0.000000,0.000000,35.000000,0.000000,19.000000,12.000000,10800.000000,4.000000,-5.000000,0.000000,3036.506212,57.213223,82.207088,14.375060,44.646428,47.417787,-18.603947
25%,2000.000000,0.000000,40.000000,3.000000,29.000000,364.000000,16560.000000,280.000000,0.000000,0.000000,12436.400419,397.170829,442.290823,424.802901,484.211476,472.282179,468.529031
50%,2000.000000,0.000000,40.000000,3.000000,36.000000,364.000000,16560.000000,825.000000,6.000000,0.000000,15808.977972,804.773034,805.293636,790.148671,808.225133,802.560892,743.961413
75%,2500.000000,0.000000,55.000000,7.000000,46.000000,547.000000,33080.000000,1200.000000,13.000000,0.000000,30127.600521,1100.751454,1144.271057,1131.725594,1181.379995,1120.673510,1154.939028
max,3500.000000,120.000000,1000.000000,60.000000,111.000000,621.000000,64484.000000,10000.000000,49.000000,1.000000,97335.243581,3650.416145,3016.353783,3823.958464,4416.075074,15261.483641,22158.957288


In [ ]:
def creat_submission(ss,test):
  for i,j in enumerate(test['m1']):
    ss.loc[i*6,'Target']=j
  for i,j in enumerate(test['m2']):
    ss.loc[(i*6)+1,'Target']=j
  for i,j in enumerate(test['m3']):
    ss.loc[(i*6)+2,'Target']=j
  for i,j in enumerate(test['m4']):
    ss.loc[(i*6)+3,'Target']=j
  for i,j in enumerate(test['m5']):
    ss.loc[(i*6)+4,'Target']=j
  for i,j in enumerate(test['m6']):
    ss.loc[(i*6)+5,'Target']=j
  return ss

In [ ]:
#create a submission file
ss=creat_submission(ss,test)

In [ ]:
#now we remove negative values and replace by 1
def denegate(ss):
  for i,j in enumerate(ss['Target']):
    if j < 0:
      ss.loc[i,'Target']=1
  return ss

In [ ]:
#test.head()

In [ ]:
ss=denegate(ss)

In [ ]:
ss.to_csv('koubrah.csv',index=False)

In [ ]:
ss.head(12)

,ID,Target
0,ID_6L67PAA x m1,837.976440
1,ID_6L67PAA x m2,909.300971
2,ID_6L67PAA x m3,1011.235254
3,ID_6L67PAA x m4,1183.289842
4,ID_6L67PAA x m5,1087.407833
5,ID_6L67PAA x m6,1225.061782
6,ID_VJ80SX2 x m1,1025.669702
7,ID_VJ80SX2 x m2,1040.329373
8,ID_VJ80SX2 x m3,1051.291827
9,ID_VJ80SX2 x m4,1079.102322
